# Setup

In [9]:
from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow.keras import(
    layers,
    Sequential,
    callbacks,
    optimizers
)

import numpy as np

In [6]:
print(f'Tensorflow version: {tf.__version__}')
assert version.parse(tf.__version__).release[0] >= 2,\
    'This notebook requires TensorFlow 2.0 or above'

Tensorflow version: 2.8.0


# Setup Data for a Simple Regression

In [8]:
data_size = 1000
train_perc = 0.8
train_size = int(data_size * train_perc)

x = np.linspace(-1, 1, data_size)
np.random.shuffle(x)

y = 0.5 * x + 2 + np.random.normal(0, 0.05, (data_size, ))

x_train, y_train = x[:train_size], y[:train_size]
x_test, y_test = x[train_size:], y[train_size:]

In [17]:
log_dir = 'logs/scalars/' + datetime.now().strftime('%Y%m%d-%H%M%S')
tensorbaord_cb = callbacks.TensorBoard(log_dir=log_dir)

model = Sequential([
    layers.Dense(16, input_dim=1),
    layers.Dense(1),
])

model.compile(
    loss = 'mse',
    optimizer = optimizers.SGD(learning_rate=0.2)
)

history = model.fit(x_train, y_train,
                    batch_size = train_size,
                    verbose=0,
                    epochs=100,
                    validation_data=[x_test, y_test],
                    callbacks=[tensorbaord_cb])

avg_loss = round(np.average(history.history['loss']), 4)

print(f'The average test loss is: {avg_loss}')

2022-03-14 12:02:57.925747: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-14 12:02:57.994180: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


The average test loss is: 0.0513


In [19]:
%load_ext tensorboard
%tensorboard --logdir logs/scalars

In [20]:
predictions = model.predict([60, 25, 2])

print(predictions)

[[31.676832]
 [14.364406]
 [ 2.98767 ]]


2022-03-14 12:10:28.364428: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


# Logging Custom Scalars

In [21]:
log_dir = 'logs/scalars/' + datetime.now().strftime('%Y%m%d-%H%M%S')
file_writer = tf.summary.create_file_writer(log_dir + '/metrics')
file_writer.set_as_default()

def lr_schedule(epoch):
    """Returns a custom learning rate that decreases as epochs progress.

    Args:
        epoch (int): The number of the current epoch
    """
    
    learning_rate = 0.2
    if epoch > 10:
        learning_rate = 0.02
    if epoch > 20:
        learning_rate = 0.01
    if epoch > 50:
        learning_rate = 0.005
        
    tf.summary.scalar('learning_rate', data=learning_rate, step=epoch)
    
    return learning_rate

lr_cb = callbacks.LearningRateScheduler(lr_schedule)
tensorbaord_cb = callbacks.TensorBoard(log_dir=log_dir)

model = Sequential([
    layers.Dense(16, input_dim=1),
    layers.Dense(1)
])

model.compile(
    loss='mse',
    optimizer=optimizers.SGD(learning_rate=0.02)
)

history = model.fit(
    x_train,
    y_train,
    batch_size=train_size,
    verbose=0,
    epochs=100,
    validation_data=(x_test, y_test),
    callbacks=[lr_cb, tensorbaord_cb]
)

2022-03-14 12:20:00.323251: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-14 12:20:00.392436: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
